# ノード操作のスケジュールを削除する
---

workerノードのスケジュール設定を削除します。

## ノード操作を行う環境の指定

VCノード操作コンテナを実行する環境のansibleグループを指定します。

既存のグループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上記のセルに表示されたグループ名から電源操作コンテナを実行しているグループ名を次のセルで指定してください。

In [ ]:
# (例)
# group = 'CoursewareHub'
# group = 'powerctl'
# group = 'scaleout'

group = 

### チェック

指定されたグループ名が妥当なものであることをチェックします。

設定されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(group)
target_group = group if "rsc_yml" not in gvars else f"{group}_manager"

指定されたノードにansibleでアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

スケジュールの設定ファイルが対象ノードに存在していることを確認します。

In [ ]:
!ansible {target_group} -a 'test -f {gvars["vcnode_ctl_dir"]}/schedule.json'

設定対象となるansibleグループにスケジュール設定があることを確認します。

In [ ]:
tgt_gvars = load_group_vars(gvars['ugroup_name'])
if "schedule" not in tgt_gvars:
    raise RuntimeError(f"{gvars['ugroup_name']}にはスケジュールの設定が行われていません")

### VCCアクセストークンの入力

VCノードの状態を確認するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなる場合は入力されたアクセストークンに誤りがあります。この節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## スケジュール設定の停止

### タイマーの停止

VCノードのスケジュール設定を行うsystemdタイマーを停止して無効化します。

現在のsystemdタイマーの設定状況を確認します。

In [ ]:
flags = '--user' if not target_group.startswith(gvars["ugroup_name"]) else ''
!ansible {target_group} -a 'systemctl {flags} list-timers'

VCノードを操作するスケジュール設定を停止して無効化します。

In [ ]:
if target_group.startswith(gvars["ugroup_name"]):
    become = '-b'
    scope = 'system'
else:
    become = ''
    scope  = 'user'

!ansible {target_group} {become} -m systemd -a \
    'name=cwh-worker-node.timer enabled=false state=stopped scope={scope}'

systemdタイマーの設定一覧を表示します。`cwh-worker-node.timer` が表示されなくなっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'systemctl {flags} list-timers'

### workerノードの構成を設定ファイルに反映する

インベントリやgroup_varsにあるworkerノードに関する情報を現在のworkerノードに合うように更新します。

現在のworkerノードに割り当てられているIPアドレスを確認します。

In [ ]:
from ipaddress import IPv4Address, AddressValueError

ug = vcp.get_ugroup(gvars["ugroup_name"])
ut = ug.get_unit("worker")
nodes = ut.find_nodes()
worker_ipaddresses = []
for node in nodes:
    try:
        worker_ipaddresses.append(str(IPv4Address(node.cloud_instance_address)))
    except AddressValueError:
        pass

print(worker_ipaddresses)

group_varsファイルのパラメータを現在のworkerノードに合うように更新します。

In [ ]:
update_group_vars(
    gvars["ugroup_name"],
    worker_ipaddresses=worker_ipaddresses,
    worker_nodes=len(worker_ipaddresses),
)

インベントリファイルの内容を現在のworkerノードに合うように更新します。

In [ ]:
import yaml
from pathlib import Path

inventory_path = Path("inventory.yml")
with inventory_path.open(encoding="utf-8") as f:
    inventory = yaml.safe_load(f)
inventory["all"]["children"][gvars["ugroup_name"]]["children"][f"{gvars['ugroup_name']}_worker"]["hosts"] = {
    x: {"servicenet_ip": x} for x in worker_ipaddresses
}
with inventory_path.open(mode="w", encoding="utf-8") as f:
    yaml.safe_dump(inventory, f)
    
!cat {inventory_path}

### group_varsの更新

group_varsファイルからスケジュール設定に関するパラメータを削除します。

In [ ]:
if target_group.startswith(gvars["ugroup_name"]):
    !cp group_vars/{group} group_vars/.{group}.bak || true
    remove_group_vars(group, "vcnode_ctl_dir", "vcnode_schedule", "schedule", "vcnode_all_ipaddress", if_exists=True)
else:
    remove_group_vars(gvars["ugroup_name"], "schedule", if_exists=True)
    !mv group_vars/{target_group} group_vars/.{target_group}.bak || true
    remove_group_from_inventory_yml(target_group)